In [1]:
import utils
import app
import json

from app import ECommerceApp

print("All packages imported\n")

# Instance of ECommerceApp 
app = ECommerceApp()
app.load_users()
print(utils.print_divider())
app.load_catalog()

print("Finished loading demo data!")

Category 'Home Appliances' added successfully.
Product 'Lamp to relax' added to category 'home_appliances' successfully.
User admin@test.com added successfully
User user@test.com added successfully
Admin admin@test.com logged in successfully!
User user@test.com logged in successfully!
Loading demo catalog from JSON file ... 

Category 'Books' added successfully.
Product 'Python Crash Course: A Hands-On, Project-Based Introduction to Programming' added to category 'books' successfully. <database.Product object at 0x11e49dde0> <class 'database.Product'>

Product 'Fluent Python: Clear, Concise, and Effective Programming' added to category 'books' successfully. <database.Product object at 0x11e49db10> <class 'database.Product'>

Product 'Introduction to Machine Learning With Python' added to category 'books' successfully. <database.Product object at 0x11e49d9c0> <class 'database.Product'>

Category 'Clothing' added successfully.
Product 'AUTOMET Womens Faux Leather Jackets' added to catego

In [2]:
print(utils.print_header("Welcome to the Demo Marketplace"))
user_session_id = app.login("maria@test.com", "user1", user_type='user')
app.view_categories()
print("\n")

Welcome to the Demo Marketplace
User maria@test.com logged in successfully!
View Categories

Category : Books
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99
  2: Fluent Python: Clear, Concise, and Effective Programming - $19.99
  3: Introduction to Machine Learning With Python - $56.99

Category : Clothing
  4: AUTOMET Womens Faux Leather Jackets - $29.99

Category : Footwear
  5: Blue running Man shoes - $29.99




In [3]:
print("Add Category & Product")
print(utils.print_divider())
# Only admins can add / update / remove categories
result = app.add_category(user_session_id, "Home Appliances", "Everything you need for your home")
if result[0] == False:
    print(result[1])
    print("Sorry, only Admins can add, update or remove products to the catalog")

Add Category & Product

Admin privileges required.
Sorry, only Admins can add, update or remove products to the catalog


In [4]:
print(utils.print_header("Admin's Dashboard"))
admin_session_id = app.login("carmen@test.com", "admin1", user_type="admin")
app.view_categories()

Admin's Dashboard
Admin carmen@test.com logged in successfully!
View Categories

Category : Books
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99
  2: Fluent Python: Clear, Concise, and Effective Programming - $19.99
  3: Introduction to Machine Learning With Python - $56.99

Category : Clothing
  4: AUTOMET Womens Faux Leather Jackets - $29.99

Category : Footwear
  5: Blue running Man shoes - $29.99


True

In [5]:
print("Add Category & Product")
print(utils.print_divider())
print(app.add_category(admin_session_id, "Home Appliances", "Everything you need for your home"))
print(app.add_product(admin_session_id,"Lamp to relax", "Himalayan Pink Lamp", 19.99, 'home_appliances'))
print(app.add_product(admin_session_id,"Lamp to concentrate", "Himalayan Blue Lamp", 19.99, 'home_appliances'))
print(app.add_product(admin_session_id, "Ruby for children", "Teaching Ruby to children", 19.99, "books", author = "Carmen Diaz"))
print(app.add_product(admin_session_id, "Teach your kids to Code", "Teaching code in a fun way", 10.00, "footwear", author = "Carmen Diaz"))

app.view_categories()

Add Category & Product

Category 'Home Appliances' added successfully.
Product 'Lamp to relax' added to category 'home_appliances' successfully.
Product 'Lamp to concentrate' added to category 'home_appliances' successfully.
Product 'Ruby for children' added to category 'books' successfully.
Product 'Teach your kids to Code' added to category 'footwear' successfully.
View Categories

Category : Books
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99
  2: Fluent Python: Clear, Concise, and Effective Programming - $19.99
  3: Introduction to Machine Learning With Python - $56.99
  8: Ruby for children - $19.99

Category : Clothing
  4: AUTOMET Womens Faux Leather Jackets - $29.99

Category : Footwear
  5: Blue running Man shoes - $29.99
  9: Teach your kids to Code - $10.0

Category : Home Appliances
  6: Lamp to relax - $19.99
  7: Lamp to concentrate - $19.99


True

In [6]:
print("Admin update/remove products. Admin add existing product")
print(utils.print_divider())
# Alreday exists
print(app.add_product(admin_session_id, "Blue running Man shoes", "Comfy shoes", 29.99, "footwear"))

# Update product's price and name
app.update_product(admin_session_id, 7, name = "Lamp to concentrate better", price = 39.99)

# Update product's category: Teach your kids to Code should be in "books"
app.update_product(admin_session_id, 9, category = "books")

# Remove product: AUTOMET Womens Faux Leather Jackets
app.remove_product(admin_session_id, 4)

app.view_categories()

Admin update/remove products. Admin add existing product

Product Blue running Man shoes already exists in footwear.
Product 'Lamp to concentrate better' updated successfully.
Product 'Teach your kids to Code' updated successfully.
Item 'AUTOMET Womens Faux Leather Jackets' removed successfully.
View Categories

Category : Books
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99
  2: Fluent Python: Clear, Concise, and Effective Programming - $19.99
  3: Introduction to Machine Learning With Python - $56.99
  8: Ruby for children - $19.99
  9: Teach your kids to Code - $10.0

Category : Clothing

Category : Footwear
  5: Blue running Man shoes - $29.99

Category : Home Appliances
  6: Lamp to relax - $19.99
  7: Lamp to concentrate better - $39.99


True

In [7]:
app.view_categories()
app.view_cart(user_session_id)


View Categories

Category : Books
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99
  2: Fluent Python: Clear, Concise, and Effective Programming - $19.99
  3: Introduction to Machine Learning With Python - $56.99
  8: Ruby for children - $19.99
  9: Teach your kids to Code - $10.0

Category : Clothing

Category : Footwear
  5: Blue running Man shoes - $29.99

Category : Home Appliances
  6: Lamp to relax - $19.99
  7: Lamp to concentrate better - $39.99
View User cart
Cart is empty.


False

In [8]:

print(utils.print_header("User adds items to cart"))
# User adds items to cart
app.add_to_cart(user_session_id, 1, 2)
app.add_to_cart(user_session_id, 3, 1)
app.add_to_cart(user_session_id, 8, 1)
app.add_to_cart(user_session_id, 7, 3)
app.view_cart(user_session_id)

User adds items to cart
Product 'Python Crash Course: A Hands-On, Project-Based Introduction to Programming' added to cart. Quantity: 2
Product 'Introduction to Machine Learning With Python' added to cart. Quantity: 1
Product 'Ruby for children' added to cart. Quantity: 1
Product 'Lamp to concentrate better' added to cart. Quantity: 3
View User cart
Items in cart:
  1: Python Crash Course: A Hands-On, Project-Based Introduction to Programming - $44.99 x 2
  3: Introduction to Machine Learning With Python - $56.99 x 1
  8: Ruby for children - $19.99 x 1
  7: Lamp to concentrate better - $39.99 x 3

Total amount $ 286.93


True

In [9]:
# Only users can add and remove to their cart
app.add_to_cart(admin_session_id, 8, 1)
app.view_cart(admin_session_id)

User privileges required.
User privileges required.


False